In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
sns.set() # Setting seaborn as default style even if use only matplotlib

df_all = pd.concat(
    [
        pd.read_csv(
            f"https://raw.githubusercontent.com/ClassicSours/TheInterstellarAsteroidRush/main/Asteroids/Asteroid%20{i}.csv"
        )
        for i in range(1, 21)
    ]
)

In [ ]:
df_buzz = df_all[df_all['DRILL_BIT_NAME'] == 'Buzz Drilldrin']
df_astro = df_all[df_all['DRILL_BIT_NAME'] == 'AstroBit']
df_apollo = df_all[df_all['DRILL_BIT_NAME'] == 'Apollo']
df_challenge = df_all[df_all['DRILL_BIT_NAME'] == 'ChallengDriller']

In [ ]:
df_all.columns

Index(['BIT_DEPTH', 'RATE_OF_PENETRATION', 'HOOK_LOAD',
       'DIFFERENTIAL_PRESSURE', 'WEIGHT_ON_BIT', 'DRILL_BIT_ID',
       'DRILL_BIT_NAME'],
      dtype='object')

In [ ]:
dfs = [df_buzz, df_astro, df_apollo, df_challenge]
for df in dfs:
  df= df.loc[~df.index.duplicated(), :]

In [ ]:
cat = {'DRILL_BIT_ID', 'DRILL_BIT_NAME'}
const = 15.0
# normalize
without_cats = []
for df in dfs:
  normalized_df = df
  normalized_df = normalized_df.loc[~normalized_df.index.duplicated(), :]
  for col in df.columns:
      if col not in cat:
          normalized_df[col] = (normalized_df[col]-normalized_df[col].mean())/normalized_df[col].std()
          Q1 = normalized_df[col].quantile(0.25)
          Q3 = normalized_df[col].quantile(0.75)
          IQR = (Q3 - Q1)    #IQR is interquartile range. 

          filter = (normalized_df[col] >= Q1 - 1.5 * IQR) & (normalized_df[col] <= Q3 + 1.5 *IQR)
          normalized_df = normalized_df[normalized_df[col] >= Q1 - const * IQR]
          normalized_df = normalized_df[normalized_df[col] <= Q3 + const * IQR]
          normalized_df = normalized_df[normalized_df[col] != 0]
  without_cats.append(normalized_df.drop(['DRILL_BIT_NAME'], axis=1))
          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import sklearn
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures
from joblib import dump, load

In [ ]:
for a in range(4):
  print(len(dfs[a]))

76489
76522
76548
73023


In [ ]:
powers = [3,2,3,2]

In [ ]:
i = 1
for without_cat in without_cats:
  power = powers[i-1]
  model = LinearRegression()
  #without_cat["COMBINE"] = without_cat["HOOK_LOAD"] * without_cat["DIFFERENTIAL_PRESSURE"]
  training_data, testing_data = sklearn.model_selection.train_test_split(without_cat, test_size=0.2, random_state=25)
  training_x = training_data.loc[:, training_data.columns!='RATE_OF_PENETRATION']
  training_y = training_data['RATE_OF_PENETRATION']
  test_x = testing_data.loc[:, testing_data.columns!='RATE_OF_PENETRATION']
  test_y = testing_data['RATE_OF_PENETRATION']
  training_x = preprocessing.scale(training_x)
  test_x = preprocessing.scale(test_x)
  training_y = preprocessing.scale(training_y)
  test_y = preprocessing.scale(test_y)
  poly = PolynomialFeatures(degree=power, include_bias=False)
  poly_features_train = poly.fit_transform(training_x)
  poly_features_test = poly.fit_transform(test_x)
  model.fit(poly_features_train, training_y)
  print(model.coef_)
  score = model.score(poly_features_test, test_y)
  print("Score for Bit Type " + str(i) + ": " + str(score))
  dump(model, 'model' + str(i) + '.joblib') 
  i += 1

Score for Bit Type 1: -0.009832493718450408
Score for Bit Type 2: 0.00046474611283098266
Score for Bit Type 3: -0.003545808163631703
Score for Bit Type 4: -0.002297794300458955


In [ ]:
from joblib import load
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

powers = [3,2,3,2]

def predict(filename, X, bit_num ): #bit num is not bit_id
  clf = load(filename)
  X = preprocessing.scale(X)
  poly = PolynomialFeatures(degree=powers[bit_num - 1], include_bias=False)
  poly_X = poly.fit_transform(X)
  return clf.predict(poly_X)